# Raz Elbaz - 315837658

# Noa Anaki - 318879491

# Git Link - https://github.com/Razelbaz1/Advanced_data_mining_and_analysis_Course

In [1]:
import requests
from bs4 import BeautifulSoup

In [2]:
url = "https://www.ad.co.il/car?sp261=13900&rp264=1980,2009"
response = requests.get(url)
if response.status_code == 200:
    print("Success")
else:
    print("Failure")



Success


In [3]:
url

'https://www.ad.co.il/car?sp261=13900&rp264=1980,2009'

In [4]:
# Function to check for the presence of a specific class on the page
def check_for_class(page_content, class_name):
    soup = BeautifulSoup(page_content, 'html.parser')
    return soup.find('div', {'class': class_name})

In [5]:
# Base URL for car listings
# add index
base_url = url +str("&pageindex=")

# Initial page index
page_index = 1

# List to store the URLs
url_list = []

while True:
    url = base_url + str(page_index)
    print(f"Fetching page: {page_index}")
    response = requests.get(url)
    
    # Check if the response contains valid content
    if response.status_code == 200:
        # Check for the specified class
        class_to_check = "d-none p-1"
        class_found = check_for_class(response.content, class_to_check)
        if class_found:
            print(f"Class '{class_to_check}' found. Stopping search.")
            break
        else:
             # Add the URL to the list
            url_list.append(url)
            #print(f"Class '{class_to_check}' not found on page {page_index}.")
    else:
        print(f"Failed to retrieve the page. Status code: {response.status_code}")
        break
    
    # Increment the page index
    page_index += 1
url_list

Fetching page: 1
Fetching page: 2
Fetching page: 3
Fetching page: 4
Fetching page: 5
Class 'd-none p-1' found. Stopping search.


['https://www.ad.co.il/car?sp261=13900&rp264=1980,2009&pageindex=1',
 'https://www.ad.co.il/car?sp261=13900&rp264=1980,2009&pageindex=2',
 'https://www.ad.co.il/car?sp261=13900&rp264=1980,2009&pageindex=3',
 'https://www.ad.co.il/car?sp261=13900&rp264=1980,2009&pageindex=4']

In [6]:
import requests
from bs4 import BeautifulSoup

def get_cars(url_list):
    cars_list = []
    
    for url in url_list:
        response = requests.get(url)
        #print(f"Fetching URL: {url}")
        
        if response.status_code != 200:
            print(f"Failed to retrieve the page. Status code: {response.status_code}")
            continue
        
        try:
            results_page = BeautifulSoup(response.content, 'html.parser')
            cars = results_page.find_all('div', {'class': 'card-block'})
            
            for car in cars:
                a_tag = car.find('a')
                if a_tag and a_tag.get('href').startswith('/ad/'):
                    title_tag = car.find('h2', {'class': 'card-title mb-0 mb-sm-1'})
                    if title_tag and "מאזדה" in title_tag.get_text():
                        car_link = "https://www.ad.co.il" + a_tag.get('href')
                        cars_list.append(car_link)
                        #print(f"Car link found: {car_link}")
        
        except Exception as e:
            print(f"An error occurred: {e}")
            continue
    
    return cars_list

In [7]:
len(get_cars(url_list))

153

In [8]:
ad_list = get_cars(url_list)
ad_list

['https://www.ad.co.il/ad/16189114',
 'https://www.ad.co.il/ad/16186152',
 'https://www.ad.co.il/ad/16167062',
 'https://www.ad.co.il/ad/16137890',
 'https://www.ad.co.il/ad/16128563',
 'https://www.ad.co.il/ad/16086340',
 'https://www.ad.co.il/ad/16045395',
 'https://www.ad.co.il/ad/16022117',
 'https://www.ad.co.il/ad/15899707',
 'https://www.ad.co.il/ad/15852178',
 'https://www.ad.co.il/ad/15835740',
 'https://www.ad.co.il/ad/15823385',
 'https://www.ad.co.il/ad/16158490',
 'https://www.ad.co.il/ad/15641462',
 'https://www.ad.co.il/ad/15512767',
 'https://www.ad.co.il/ad/15485521',
 'https://www.ad.co.il/ad/15485589',
 'https://www.ad.co.il/ad/15392534',
 'https://www.ad.co.il/ad/15350355',
 'https://www.ad.co.il/ad/15218480',
 'https://www.ad.co.il/ad/15080625',
 'https://www.ad.co.il/ad/15038525',
 'https://www.ad.co.il/ad/16111566',
 'https://www.ad.co.il/ad/14986594',
 'https://www.ad.co.il/ad/14951318',
 'https://www.ad.co.il/ad/14940631',
 'https://www.ad.co.il/ad/14868498',
 

### Create Dataframe :

In [9]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from datetime import datetime

def extract_car_details(url):
    response = requests.get(url)
    if response.status_code != 200:
        print(f"Failed to retrieve the page. Status code: {response.status_code}")
        return None

    try:
        page_content = BeautifulSoup(response.content, 'html.parser')
        
        # Initialize a dictionary to store car details
        car_details = {
            "manufacturer": None,
            "Year": None,
            "model": None,
            "Hand": None,
            "Gear": None,
            "capacity_Engine": None,
            "Engine_type": None,
            "Prev_ownership": None,
            "Curr_ownership": None,
            "Area": None,
            "City": None,
            "Price": None,
            "pic_num": None,
            "Cre_date": None,
            "Repub_date": None,
            "Description": None,
            "Color": None,
            "Km": None,
            "Test": None
        }

        # Extract details from the table
        table = page_content.find('table', {'class': 'table'})
        if table:
            rows = table.find_all('tr')
            for row in rows:
                columns = row.find_all('td')
                if len(columns) == 2:
                    key = columns[0].get_text().strip()
                    value = columns[1].get_text().strip()
                    if key == 'טסט עד':  # If the key is 'test until', extract the test expiration date
                        try:
                            test_date = datetime.strptime(value, '%m/%Y')
                            today = datetime.today()
                            time_until_test = test_date - today
                            car_details["Test"] = int(time_until_test.days) if int(time_until_test.days) >= 0 else None
                        except ValueError:
                            car_details["Test"] = None

        # Extract manufacturer and model
        title = page_content.find('h2', {'class': 'card-title'})
        if title:
            title_text = title.get_text().strip().split()
            if len(title_text) > 1:
                car_details["manufacturer"] = title_text[0]
                car_details["model"] = ' '.join(title_text[1:])
        
        # Extract Price
        prices = page_content.find_all('h2', {'class': 'card-title'})
        if prices:
            try:
                price = prices[1].get_text().strip().split()[0]
                car_details["Price"] = float(price.replace('₪', '').replace(',', ''))
            except (IndexError, ValueError):
                car_details["Price"] = None

        # Extract Year
        year = page_content.find('p', {'class': 'card-text'})
        if year:
            try:
                car_details["Year"] = int(year.get_text().strip())
            except ValueError:
                car_details["Year"] = None

        # Extract other details from the table
        if table:
            for row in rows:
                columns = row.find_all('td')
                if len(columns) == 2:
                    key = columns[0].get_text().strip()
                    value = columns[1].get_text().strip()
                    try:
                        if key == 'שנה':
                            car_details["Year"] = int(value)
                        elif key == 'יד':
                            car_details["Hand"] = int(value)
                        elif key == 'ת. הילוכים':
                            car_details["Gear"] = value
                        elif key == 'נפח':
                            car_details["capacity_Engine"] = int(value.replace(',', ''))
                        elif key == 'סוג מנוע':
                            car_details["Engine_type"] = value
                        elif key == 'ק"מ':
                            car_details["Km"] = int(value.replace(',', ''))
                        elif key == 'צבע':
                            car_details["Color"] = value
                        elif key == 'בעלות קודמת':
                            car_details["Prev_ownership"] = value
                        elif key == 'בעלות נוכחית':
                            car_details["Curr_ownership"] = value
                        elif key == 'אזור':
                            car_details["Area"] = value
                        elif key == 'עיר':
                            car_details["City"] = value
                    except ValueError as e:
                        print(f"Error processing key {key} with value {value}: {e}")
                        continue
        
        # Extract number of images
        image_container = page_content.find('div', class_='col-12 d-flex mt-3 justify-content-center flex-wrap')
        if image_container:
            images = image_container.find_all('img')
            car_details["pic_num"] = int(len(images))
        
        # Extract Ad creation and last republication dates
        ad_dates = page_content.find('div', class_='d-flex flex-row align-items-center justify-content-center flex-wrap')
        if ad_dates:
            ad_text = ad_dates.get_text().strip()
            ad_lines = ad_text.split('\n')
            for line in ad_lines:
                try:
                    if 'תאריך יצירה' in line:
                        Create_date_str = line.split(': ')[1]
                        car_details["Cre_date"] = datetime.strptime(Create_date_str, '%d/%m/%Y')
                    elif 'תאריך הקפצה אחרון' in line:
                        repub_date_str = line.split(': ')[1]
                        car_details["Repub_date"] = datetime.strptime(repub_date_str, '%d/%m/%Y')
                except ValueError as e:
                    print(f"Error processing date line {line}: {e}")
                    continue
        
        # Extract Description
        Description = page_content.find('p', {'class': 'text-word-break'})
        if Description:
            description_text = Description.get_text().strip().replace("\n", " ")
            car_details["Description"] = description_text
        
        return car_details

    except Exception as e:
        print(f"An error occurred while processing the page {url}: {e}")
        return None

def get_car_data(url_list):
    car_data = []
    for url in url_list:
        details = extract_car_details(url)
        if details:
            car_data.append(details)
    
    # Create a DataFrame from the list of dictionaries
    df = pd.DataFrame(car_data)
    # Casting columns
    if 'pic_num' in df:
        df['pic_num'] = df['pic_num'].fillna(0).astype(int)
    df['Gear'] = df['Gear'].astype('category')
    df['Engine_type'] = df['Engine_type'].astype('category')
    df['Prev_ownership'] = df['Prev_ownership'].astype('category')
    df['Curr_ownership'] = df['Curr_ownership'].astype('category')
    df['Area'] = df['Area'].astype('string')
    df['City'] = df['City'].astype('string')
    df['Cre_date'] = df['Cre_date'].astype('datetime64')
    df['Repub_date'] = df['Repub_date'].astype('datetime64')
    df['Description'] = df['Description'].astype('string')
    df['Color'] = df['Color'].astype('string')

    # Fill NaN values in 'Km' and 'Test' with 0 and cast to int
    if 'capacity_Engine' in df:
        df['capacity_Engine'] = df['capacity_Engine'].astype(pd.Int64Dtype())
    if 'Km' in df:
        df['Km'] = df['Km'].astype(pd.Int64Dtype())
    if 'Test' in df:
        df['Test'] = df['Test'].astype(pd.Int64Dtype())
    
    return df

# Example usage
#ad_list = [
#    'https://www.ad.co.il/ad/16186152',
# 'https://www.ad.co.il/ad/16185806',
# 'https://www.ad.co.il/ad/16186411',
# 'https://www.ad.co.il/ad/16168929',
#'https://www.ad.co.il/ad/16158490'
#]


car_df = get_car_data(ad_list)
car_df


,manufacturer,Year,model,Hand,Gear,capacity_Engine,Engine_type,Prev_ownership,Curr_ownership,Area,City,Price,pic_num,Cre_date,Repub_date,Description,Color,Km,Test
0,מאזדה,2007,3,4,אוטומטית,1600,בנזין,פרטית,פרטית,באר שבע והסביבה,אופקים,8000.0,3,2024-06-05,2024-06-05,‏‪‪הרכב במצב טוב עבר טיפול לפני חודשיים לקחת ו...,אפור,189000,109
1,מאזדה,2008,6,3,אוטומטית,2000,בנזין,פרטית,פרטית,בית שמש והסביבה,בית שמש,16000.0,4,2024-05-16,2024-06-04,מאזדה 6 שנת 2008 במצב מצוין מכאנית טסט עד סוף ...,שחור,195000,109
2,מאזדה,2007,3,4,אוטומטית,1600,בנזין,פרטית,פרטית,גדרה יבנה והסביבה,יבנה,8700.0,8,2024-04-04,2024-04-04,אחלה אוטו נוסע מצוין ידיים פרטיות צמיגים במצב ...,בז' מטאלי,188000,48
3,מאזדה,2008,3,5,אוטומטית,1600,בנזין,פרטית,פרטית,מודיעין והסביבה,מודיעין מכבים רעות,12300.0,4,2024-03-16,2024-03-17,למכירה מאשה 3האצבק 2008 מנוע 1600.105 כוח סוס ...,שחור,<NA>,<NA>
4,מאזדה,2007,3,6,אוטומטית,1600,בנזין,השכרה,פרטית,חולון - בת ים,חולון,7800.0,3,2024-03-10,2024-03-13,"רכב מתוחזק עבר טיפול גדול ,הוחלף מצבר ,צמיגיפ,...",זהב,305000,48
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148,מאזדה,2008,6,7,אוטומטית,2000,בנזין,NaN,NaN,חולון - בת ים,חולון,18000.0,0,2022-04-25,2022-04-25,עבר טיפול גדול עבר שיפוץ גיר 4 גלגלים חדשים טס...,<NA>,188009,<NA>
149,מאזדה,2009,3,3,אוטומטית,2000,בנזין,פרטית,פרטית,בקעת אונו,אור יהודה,27000.0,0,2022-04-13,2022-04-13,רכב שמור טיפולים בזמן.סאנרוף נסגר עם הנעילה או...,שחור,189000,<NA>
150,מאזדה,2006,5,4,אוטומטית,2000,בנזין,פרטית,פרטית,רמת גן - גבעתיים,בני ברק,4000.0,0,2021-10-26,2021-10-25,רכב טוב יש לו בעיה שהמנוע נכבה ברמזורים וכדומה,כחול,200000,<NA>
151,מאזדה,2006,5,4,אוטומטית,2000,בנזין,פרטית,פרטית,רמת גן - גבעתיים,בני ברק,5000.0,0,2021-10-18,2021-10-18,הרכב טעון שיפוץ מנוע\גיר עקב שנכבה ברמזורים וכ...,כחול,200000,<NA>


---
### Check type.

In [10]:
car_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 153 entries, 0 to 152
Data columns (total 19 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   manufacturer     153 non-null    object        
 1   Year             153 non-null    int64         
 2   model            153 non-null    object        
 3   Hand             153 non-null    int64         
 4   Gear             153 non-null    category      
 5   capacity_Engine  152 non-null    Int64         
 6   Engine_type      153 non-null    category      
 7   Prev_ownership   54 non-null     category      
 8   Curr_ownership   54 non-null     category      
 9   Area             134 non-null    string        
 10  City             153 non-null    string        
 11  Price            148 non-null    float64       
 12  pic_num          153 non-null    int32         
 13  Cre_date         153 non-null    datetime64[ns]
 14  Repub_date       153 non-null    datetime6

In [15]:
def export_to_csv(df, file_name):
    df.to_csv(file_name, index=False)
    print(f"Data exported to {file_name}")

export_to_csv(car_df, 'car_data.csv')

Data exported to car_data.csv
